# Clustering

In [8]:
# intilializations
spark_version = "2.4.7"
spark_home = "file:///Users/kaustuv/spark-2.4.7/"
spark_outfolder = "file:///Users/kaustuv/Documents/Courses/Spark/modelout/"

from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

spark = SparkSession \
    .builder \
    .appName("StructuredNetworkWordCount") \
    .getOrCreate()

# SparkSession  to SparkContext
sc = spark.sparkContext

## K-means

- Clustering algorithms that clusters the data points into a predefined number of clusters
- MLlib implementation includes a parallelized variant of the k-means++ method called kmeans

In [7]:
# for DF API 

from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# Loads data.
dataset = spark.read.format("libsvm").load(spark_home +"data/mllib/sample_kmeans_data.txt")

# Trains a k-means model.
kmeans = KMeans().setK(2).setSeed(1)
model = kmeans.fit(dataset)

# Make predictions
predictions = model.transform(dataset)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)
    

Silhouette with squared euclidean distance = 0.9997530305375207
Cluster Centers: 
[0.1 0.1 0.1]
[9.1 9.1 9.1]


<B> K Means : RDD API</B> <br>
In the following example after loading and parsing data, we use the KMeans object to cluster the data into two clusters. The number of desired clusters is passed to the algorithm. We then compute Within Set Sum of Squared Error (WSSSE). You can reduce this error measure by increasing k. In fact the optimal k is usually one where there is an “elbow” in the WSSSE graph.

In [ ]:
from numpy import array
from math import sqrt


from pyspark.mllib.clustering import KMeans, KMeansModel

# Load and parse the data
data = sc.textFile(spark_home + "file:///Users/kaustuv/spark-3.1.1/data/mllib/kmeans_data.txt")
parsedData = data.map(lambda line: array([float(x) for x in line.split(' ')]))

# Build the model (cluster the data)
clusters = KMeans.train(parsedData, 2, maxIterations=10, initializationMode="random")

# Evaluate clustering by computing Within Set Sum of Squared Errors
def error(point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

WSSSE = parsedData.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))

# Save and load model
clusters.save(sc, "KMeansModel")
sameModel = KMeansModel.load(sc, "file:///Users/kaustuv/Documents/Courses/Spark/modelout/KMeansModel")

 ## Latent Dirichlet allocation (LDA)
 
 - Implemented as an Estimator  that generates a LDAModel as the base model

In [11]:
# DF -API 
from pyspark.ml.clustering import LDA

# Loads data.
dataset = spark.read.format("libsvm").load(spark_home +"data/mllib/sample_lda_libsvm_data.txt")

# Trains a LDA model.
lda = LDA(k=10, maxIter=10)
model = lda.fit(dataset)

ll = model.logLikelihood(dataset)
lp = model.logPerplexity(dataset)
print("The lower bound on the log likelihood of the entire corpus: " + str(ll))
print("The upper bound on perplexity: " + str(lp))

# Describe topics.
topics = model.describeTopics(3)
print("The topics described by their top-weighted terms:")
topics.show(truncate=False)

# Shows the result
transformed = model.transform(dataset)
transformed.show(truncate=False)

The lower bound on the log likelihood of the entire corpus: -780.766696363605
The upper bound on perplexity: 3.0029488321677116
The topics described by their top-weighted terms:
+-----+-----------+---------------------------------------------------------------+
|topic|termIndices|termWeights                                                    |
+-----+-----------+---------------------------------------------------------------+
|0    |[9, 7, 3]  |[0.1098239065792031, 0.09816055275801179, 0.09771350482284155] |
|1    |[8, 10, 9] |[0.11038608188739116, 0.10512054014527725, 0.09976959179450495]|
|2    |[5, 4, 0]  |[0.1612974535771344, 0.14632933298698012, 0.14596355786974397] |
|3    |[3, 9, 10] |[0.29100982941222525, 0.12431457341381975, 0.1126043709146982] |
|4    |[6, 10, 9] |[0.21142376712301392, 0.18091236002470296, 0.130266170189281]  |
|5    |[4, 1, 7]  |[0.15003575357542334, 0.14341479452923464, 0.11896759355263571]|
|6    |[3, 8, 9]  |[0.09868772347307755, 0.0980497534865028, 0.094

<b> RDD API  :</b>In the following example, we load word count vectors representing a corpus of documents. We then use LDA to infer three topics from the documents. The number of desired clusters is passed to the algorithm. We then output the topics, represented as probability distributions over words.

In [13]:
# RDD API 
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.linalg import Vectors

# Load and parse the data
data = sc.textFile(spark_home + "data/mllib/sample_lda_data.txt")
parsedData = data.map(lambda line: Vectors.dense([float(x) for x in line.strip().split(' ')]))
# Index documents with unique IDs
corpus = parsedData.zipWithIndex().map(lambda x: [x[1], x[0]]).cache()

# Cluster the documents into three topics using LDA
ldaModel = LDA.train(corpus, k=3)

# Output topics. Each is a distribution over words (matching word count vectors)
print("Learned topics (as distributions over vocab of " + str(ldaModel.vocabSize())
      + " words):")
topics = ldaModel.topicsMatrix()
for topic in range(3):
    print("Topic " + str(topic) + ":")
    for word in range(0, ldaModel.vocabSize()):
        print(" " + str(topics[word][topic]))

# Save and load model
ldaModel.save(sc, spark_outfolder + "LDAModel")
sameModel = LDAModel.load(sc, spark_outfolder +"LDAModel")


Learned topics (as distributions over vocab of 11 words):
Topic 0:
 7.136567545349074
 9.17423739769681
 2.2973332755243874
 7.74896206977181
 3.667920066552916
 2.7218615703304714
 14.290818208156182
 1.7005462721131068
 4.786929613293138
 10.88227104239743
 23.420456894402445
Topic 1:
 9.334046356900249
 6.2571239053539465
 1.889322674338553
 25.143833504691937
 13.074261767726298
 7.221478143169692
 6.281845744850988
 3.543274325089574
 1.582847027410369
 7.365571669744681
 4.559958551562936
Topic 2:
 9.529386097750677
 13.568638696949243
 7.813344050137059
 7.107204425536252
 8.257818165720789
 12.056660286499838
 10.427336046992828
 4.7561794027973185
 1.630223359296492
 5.75215728785789
 5.019584554034619


## Bisecting k-means

- Bisecting k-means is a kind of hierarchical clustering using a divisive (or “top-down”) approach: all observations start in one cluster, and splits are performed recursively as one moves down the hierarchy.

- Bisecting K-means can often be much faster than regular K-means, but it will generally produce a different clustering.

In [14]:
# DF API 
from pyspark.ml.clustering import BisectingKMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# Loads data.
dataset = spark.read.format("libsvm").load(spark_home+"data/mllib/sample_kmeans_data.txt")

# Trains a bisecting k-means model.
bkm = BisectingKMeans().setK(2).setSeed(1)
model = bkm.fit(dataset)

# Make predictions
predictions = model.transform(dataset)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

# Shows the result.
print("Cluster Centers: ")
centers = model.clusterCenters()
for center in centers:
    print(center)

Silhouette with squared euclidean distance = 0.9997530305375207
Cluster Centers: 
[0.1 0.1 0.1]
[9.1 9.1 9.1]


In [15]:
# RDD -API 
from numpy import array

from pyspark.mllib.clustering import BisectingKMeans

# Load and parse the data
data = sc.textFile(spark_home + "data/mllib/kmeans_data.txt")
parsedData = data.map(lambda line: array([float(x) for x in line.split(' ')]))

# Build the model (cluster the data)
model = BisectingKMeans.train(parsedData, 2, maxIterations=5)

# Evaluate clustering
cost = model.computeCost(parsedData)
print("Bisecting K-means Cost = " + str(cost))

Bisecting K-means Cost = 0.11999999999994547


## Gaussian Mixture Model (GMM)
-  Gaussian Mixture Model represents a composite distribution whereby points are drawn from one of k Gaussian sub-distributions, each with its own probability

- spark.ml implementation uses the expectation-maximization algorithm to induce the maximum-likelihood model given a set of samples.


In [16]:
# DF API 
from pyspark.ml.clustering import GaussianMixture

# loads data
dataset = spark.read.format("libsvm").load(spark_home + "data/mllib/sample_kmeans_data.txt")

gmm = GaussianMixture().setK(2).setSeed(538009335)
model = gmm.fit(dataset)

print("Gaussians shown as a DataFrame: ")
model.gaussiansDF.show(truncate=False)

Gaussians shown as a DataFrame: 
+-------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|mean                                                         |cov                                                                                                                                                                                                     |
+-------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[0.10000000000001552,0.10000000000001552,0.10000000000001552]|0.006666666666806454  0.006666666666806454  0.006666666666806454  
0.006666666666806454  0.006666666666806454

- A Gaussian Mixture Model represents a composite distribution whereby points are drawn from one of k Gaussian sub-distributions, each with its own probability. The spark.mllib implementation uses the expectation-maximization algorithm to induce the maximum-likelihood model given a set of samples. The implementation has the following parameters:

    k is the number of desired clusters.
    convergenceTol is the maximum change in log-likelihood at which we consider convergence achieved.
    maxIterations is the maximum number of iterations to perform without reaching convergence.
    initialModel is an optional starting point from which to start the EM algorithm. If this parameter is omitted, a random starting point will be constructed from the data.


- In the following example after loading and parsing data, we use a GaussianMixture object to cluster the data into two clusters. The number of desired clusters is passed to the algorithm. We then output the parameters of the mixture model.

In [18]:
# RDD API 
from numpy import array

from pyspark.mllib.clustering import GaussianMixture, GaussianMixtureModel

# Load and parse the data
data = sc.textFile(spark_home + "data/mllib/gmm_data.txt")
parsedData = data.map(lambda line: array([float(x) for x in line.strip().split(' ')]))

# Build the model (cluster the data)
gmm = GaussianMixture.train(parsedData, 2)

# Save and load model
gmm.save(sc, spark_outfolder + "GaussianMixtureModel")
sameModel = GaussianMixtureModel\
    .load(sc, spark_outfolder +"GaussianMixtureModel")

# output parameters of model
for i in range(2):
    print("weight = ", gmm.weights[i], "mu = ", gmm.gaussians[i].mu,
          "sigma = ", gmm.gaussians[i].sigma.toArray())

weight =  0.4803341198536062 mu =  [0.07222436012684245,0.01667856414580816] sigma =  [[4.78126493 1.87708401]
 [1.87708401 0.91494249]]
weight =  0.5196658801463938 mu =  [-0.10439917402842634,0.04285392575445832] sigma =  [[ 4.90562922 -2.00593205]
 [-2.00593205  1.01114923]]


## Power Iteration Clustering (PIC)

-  a scalable graph clustering algorithm developed by Lin and Cohen.
- PIC finds a very low-dimensional embedding of a dataset using truncated power iteration on a normalized pair-wise similarity matrix of the data.

- spark.ml’s PowerIterationClustering implementation takes the following parameters:
    - k: the number of clusters to create
    - initMode: param for the initialization algorithm
    - maxIter: param for maximum number of iterations
    - srcCol: param for the name of the input column for source vertex IDs
    - dstCol: name of the input column for destination vertex IDs
    - weightCol: Param for weight column name



- Power iteration clustering (PIC) is a scalable and efficient algorithm for clustering vertices of a graph given pairwise similarities as edge properties, described in Lin and Cohen, Power Iteration Clustering. It computes a pseudo-eigenvector of the normalized affinity matrix of the graph via power iteration and uses it to cluster vertices. spark.mllib includes an implementation of PIC using GraphX as its backend. It takes an RDD of (srcId, dstId, similarity) tuples and outputs a model with the clustering assignments. The similarities must be nonnegative. PIC assumes that the similarity measure is symmetric. A pair (srcId, dstId) regardless of the ordering should appear at most once in the input data. If a pair is missing from input, their similarity is treated as zero. spark.mllib’s PIC implementation takes the following (hyper-)parameters:
   - k: number of clusters
   - maxIterations: maximum number of power iterations
   - initializationMode: initialization model. This can be either “random”, which is the default, to use a random vector as vertex properties, or “degree” to use normalized sum similarities.


In [19]:
from pyspark.ml.clustering import PowerIterationClustering

df = spark.createDataFrame([
    (0, 1, 1.0),
    (0, 2, 1.0),
    (1, 2, 1.0),
    (3, 4, 1.0),
    (4, 0, 0.1)
], ["src", "dst", "weight"])

pic = PowerIterationClustering(k=2, maxIter=20, initMode="degree", weightCol="weight")

# Shows the cluster assignment
pic.assignClusters(df).show()

+---+-------+
| id|cluster|
+---+-------+
|  4|      1|
|  0|      0|
|  1|      0|
|  2|      0|
|  3|      1|
+---+-------+



<b> RDD API : </b><br>
PowerIterationClustering implements the PIC algorithm. It takes an RDD of (srcId: Long, dstId: Long, similarity: Double) tuples representing the affinity matrix. Calling PowerIterationClustering.run returns a PowerIterationClusteringModel, which contains the computed clustering assignments.

In [20]:
# RDD API
from pyspark.mllib.clustering import PowerIterationClustering, PowerIterationClusteringModel

# Load and parse the data
data = sc.textFile( spark_home + "data/mllib/pic_data.txt")
similarities = data.map(lambda line: tuple([float(x) for x in line.split(' ')]))

# Cluster the data into two classes using PowerIterationClustering
model = PowerIterationClustering.train(similarities, 2, 10)

model.assignments().foreach(lambda x: print(str(x.id) + " -> " + str(x.cluster)))

# Save and load model
model.save(sc, spark_outfolder + "PICModel")
sameModel = PowerIterationClusteringModel\
    .load(sc, spark_outfolder + "PICModel")

### Streaming k-means

When data arrive in a stream, we may want to estimate clusters dynamically, updating them as new data arrive. spark.mllib provides support for streaming k-means clustering, with parameters to control the decay (or “forgetfulness”) of the estimates. The algorithm uses a generalization of the mini-batch k-means update rule. For each batch of data, we assign all points to their nearest cluster, compute new cluster centers, then update each cluster 

In [21]:
from pyspark.streaming import StreamingContext

# Create a local StreamingContext with two working thread and batch interval of 1 second
#sc = SparkContext("local[2]", "NetworkWordCount")
ssc = StreamingContext(sc, 1)

In [22]:
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.clustering import StreamingKMeans

# we make an input stream of vectors for training,
# as well as a stream of vectors for testing
def parse(lp):
    label = float(lp[lp.find('(') + 1: lp.find(')')])
    vec = Vectors.dense(lp[lp.find('[') + 1: lp.find(']')].split(','))

    return LabeledPoint(label, vec)

trainingData = sc.textFile(spark_home+ "data/mllib/kmeans_data.txt")\
    .map(lambda line: Vectors.dense([float(x) for x in line.strip().split(' ')]))

testingData = sc.textFile(spark_home +"data/mllib/streaming_kmeans_data_test.txt").map(parse)

trainingQueue = [trainingData]
testingQueue = [testingData]

trainingStream = ssc.queueStream(trainingQueue)
testingStream = ssc.queueStream(testingQueue)

# We create a model with random clusters and specify the number of clusters to find
model = StreamingKMeans(k=2, decayFactor=1.0).setRandomCenters(3, 1.0, 0)

# Now register the streams for training and testing and start the job,
# printing the predicted cluster assignments on new data points as they arrive.
model.trainOn(trainingStream)

result = model.predictOnValues(testingStream.map(lambda lp: (lp.label, lp.features)))
result.pprint()

ssc.start()
ssc.stop(stopSparkContext=True, stopGraceFully=True)

-------------------------------------------
Time: 2021-06-14 20:38:12
-------------------------------------------
(1.0, 0)
(2.0, 1)

-------------------------------------------
Time: 2021-06-14 20:38:13
-------------------------------------------



## Dimensionality Reduction

### Singular value decomposition (SVD)

In [29]:
# from pyspark.mllib.linalg import Vectors
# from pyspark.mllib.linalg.distributed import RowMatrix

# rows = sc.parallelize([
#     Vectors.sparse(5, {1: 1.0, 3: 7.0}),
#     Vectors.dense(2.0, 0.0, 3.0, 4.0, 5.0),
#     Vectors.dense(4.0, 0.0, 0.0, 6.0, 7.0)
# ])

# mat = RowMatrix(rows)

# # Compute the top 5 singular values and corresponding singular vectors.
# svd = mat.computeSVD(5, computeU=True)
# U = svd.U       # The U factor is a RowMatrix.
# s = svd.s       # The singular values are stored in a local dense vector.
# V = svd.V       # The V factor is a local dense matrix.

### Principal component analysis (PCA)

- Principal component analysis (PCA) is a statistical method to find a rotation such that the first coordinate has the largest variance possible, and each succeeding coordinate, in turn, has the largest variance possible. The columns of the rotation matrix are called principal components. PCA is used widely in dimensionality reduction.

- spark.mllib supports PCA for tall-and-skinny matrices stored in row-oriented format and any Vectors.

In [28]:
# from pyspark.mllib.linalg import Vectors
# from pyspark.mllib.linalg.distributed import RowMatrix

# rows = sc.parallelize([
#     Vectors.sparse(5, {1: 1.0, 3: 7.0}),
#     Vectors.dense(2.0, 0.0, 3.0, 4.0, 5.0),
#     Vectors.dense(4.0, 0.0, 0.0, 6.0, 7.0)
# ])

# mat = RowMatrix(rows)
# # Compute the top 4 principal components.
# # Principal components are stored in a local dense matrix.
# pc = mat.computePrincipalComponents(4)

# # Project the rows to the linear space spanned by the top 4 principal components.
# projected = mat.multiply(pc)